In [12]:
import pandas as pd
import json

In [16]:
session = pd.read_csv('counter_narrative/data/train.csv')
seed = pd.read_csv('counter_narrative/data/seed_data.csv')
background = pd.read_csv('cindy/background/all_background.csv')

In [17]:
session_merged = pd.merge(session, background, on='filename', how='left')
session_merged.drop(columns=['text_x'], inplace=True)
session_merged.rename(columns={'text_y': 'text'}, inplace=True)

In [18]:
session_merged.to_csv('counter_narrative/data/session_merged.csv', index=False, encoding='utf-8')

In [20]:
test= pd.read_csv('counter_narrative/data/test.csv', encoding='utf-8')
test_merged = pd.merge(test, background, on='filename', how='left')
test_merged.drop(columns=['text_x'], inplace=True)
test_merged.rename(columns={'text_y': 'text'}, inplace=True)
test_merged.to_csv('counter_narrative/data/test_gen_merged.csv', index=False, encoding='utf-8')

In [6]:
seed_merged = pd.merge(seed, background, on='filename', how='left')
seed_merged.drop(columns=['text_x','label'], inplace=True)
seed_handle = seed_merged.to_dict(orient='records')

In [7]:
session = pd.merge(session, background, on='filename', how='left')
session.drop(columns=['text_x','label'], inplace=True)
session = session.to_dict(orient='records')

In [8]:
SEP = "<<<SEP>>>"

seed_list = []
for item in seed_handle:
    contents = (
        f"[HS] {item['text_y']}{SEP}"
        f"[KNOWLEDGE] {item['background']}{SEP}"
        f"[TYPE] Positive Response{SEP}[CN] {item['Positive Response']}{SEP}"
        f"[TYPE] Counter Question{SEP}[CN] {item['Counter Question']}{SEP}"
        f"[TYPE] Empathy-based Counter Narrative{SEP}[CN] {item['Empathy-based Counter Narrative']}{SEP}"
        f"[TYPE] Warning/Denouncing Response{SEP}[CN] {item['Warning/Denouncing Response']}"
    )
    seed_list.append(contents)

for row in session:
    contents = (
        f"[HS] {row['text_y']}{SEP}"
        f"[KNOWLEDGE] {row['background']}{SEP}"
        f"[TYPE] {row['counter_type']}{SEP}"
        f"[CN] {row['counter_text']}"
    )
    seed_list.append(contents)


In [9]:
seed_list

['[HS] 如果结婚是好事的话 按照我国的国情 应该托关系走后门才能结婚<<<SEP>>>[KNOWLEDGE] 这段话通过讽刺的语气表达了对婚姻制度或社会现状的不满，暗示结婚并不如宣传的那样美好。提到“托关系走后门”反映了对社会中潜在不公正现象的批评。整体情感基调是调侃和无奈。<<<SEP>>>[TYPE] Positive Response<<<SEP>>>[CN] \u2060婚姻本该是一种基于平等、尊重和自由选择的关系，而不是被社会强加给女性的枷锁。如果能真正做到互相支持、共同成长，婚姻也可以是女性选择的幸福生活方式之一，而不是被迫接受的“任务”。<<<SEP>>>[TYPE] Counter Question<<<SEP>>>[CN] 我们是否应该先问问，为什么在很多人眼中，婚姻不再是幸福的象征，而成了一种负担甚至牺牲？<<<SEP>>>[TYPE] Empathy-based Counter Narrative<<<SEP>>>[CN] 我能理解你的感受，很多女性在婚姻中确实会面临被压迫、失去独立性的现实困境。这也是为什么越来越多人呼吁尊重女性的选择权，让婚姻成为一种自愿而非必须的选择。<<<SEP>>>[TYPE] Warning/Denouncing Response<<<SEP>>>[CN] 如果我们继续把婚姻看作女性必须完成的任务，忽视她们的个人自由和发展，这不仅会让更多人恐惧婚姻，也会加深社会对女性的控制和物化，最终伤害所有人。',
 '[HS] 为什么不生孩子？ 让男的生呗 他们不是先生嘛？<<<SEP>>>[KNOWLEDGE] 这段话带有讽刺和反问的语气，质疑传统性别角色和生育责任。通过调侃“先生”一词，表达了对男性在生育问题上缺乏实际责任的批评。整体情感上有不满和对性别不平等的反思。<<<SEP>>>[TYPE] Positive Response<<<SEP>>>[CN] 生孩子应该是每个人的自由选择，不该被强迫或道德绑架。每个人，无论性别，都有权利决定自己的身体和生活方式，社会应该尊重这样的选择。<<<SEP>>>[TYPE] Counter Question<<<SEP>>>[CN] 为什么总是把生育的责任推给女性？是不是该先问问，为什么社会默认生育是“女性必须”而不是“共同选择”？<<<SEP>>>[TYPE] Empathy

In [10]:
import json
from pathlib import Path

out_file = Path("seed_triple.jsonl").open("w", encoding="utf8")

for i, row in enumerate(seed_list):
    doc = {
        "id": f"doc_{i:06d}",
        "contents":row,
    }
    out_file.write(json.dumps(doc, ensure_ascii=False) + "\n")

out_file.close()
print("✅ JSONL 已写出：", i + 1, "条")


✅ JSONL 已写出： 449 条


In [56]:
import json, pathlib
path = pathlib.Path('seed_triple.jsonl')
for ln, line in enumerate(path.open(encoding='utf8'), 1):
    try:
        obj = json.loads(line)
        assert 'id' in obj and 'contents' in obj
    except Exception as e:
        print(f'⚠️  第 {ln} 行格式问题:', e)
        break
else:
    print('✅ 全部合法，共', ln, '行')


✅ 全部合法，共 449 行
